<figure>
    <img src="../../../figures/logo_ap.png"  width="80" height="80" align="left"/>
</figure>

# <span style="color:blue"><blue><br><br><br><br><center>&nbsp;&nbsp;&nbsp;Aprendizaje Profundo</center></span>

# <span style="color:red"><center>Biblioteca Alejandría</center></span>

## <span style="color:green"><center>Scraping Web de Fuentes Académicas con Agentes</center></span>

## <span style="color:blue">Autores</span>

1. Álvaro Montenegro, alvaro.montenegro@aprendizajeprofundo.ai
1. Daniel Montenegro, daniel.montenegro@aprendizajeprofundo.ai

## <span style="color:blue">Contenido</span>

* [Librerías Necesarias](#Librerías-Necesarias)
* [Realizar Consultas](#Realizar-Consultas)
* [Extraer Metadata](#Extraer-Metadata)
* [Convertir en DataFrame](#Convertir-en-DataFrame)
    * [Subsección X](#Subsección-X)
* [Automatización](#Automatización)
* [Conclusiones](#Conclusiones)
* [Recomendaciones](#Recomendaciones)

## <span style="color:blue">Librerías Necesarias</span>

In [4]:
import os
import urllib
import autogen
import pandas as pd
from dotenv import load_dotenv
from serpapi import GoogleSearch
from autogen import AssistantAgent, UserProxyAgent

load_dotenv("../../../.env")

True

[[Volver]](#Contenido)

## <span style="color:blue">Realizar Consultas</span>

Texto de la sección X

In [3]:
from typing import Annotated
import requests

def fetch_html(
    url: Annotated[str, "La URL a la que se hará la solicitud"]
) -> str:
    """
    Realiza una solicitud HTTP GET usando urllib y devuelve el contenido HTML.
    """
    try:
        # Realizar la solicitud
        data = requests.get(url)
        # Leer y decodificar el contenido
        html = data.text
        return html
    except Exception as e:
        # Manejo básico de errores
        return f"Error al hacer la solicitud: {str(e)}"

# Ejemplo de uso
type_query = "all"
query = "RAG"
start = 0
max_results = 1
sortby = "submittedDate"
sortorder = "descending"
url = f'http://export.arxiv.org/api/query?search_query={type_query}:{query}&start={start}&max_results={max_results}&sortBy={sortby}&sortOrder={sortorder}'
url = f"https://towardsdatascience.com/search?q={query}"

html_content = fetch_html(url=url)
#print(html_content)

In [5]:
import os
from autogen import ConversableAgent

### SHOULD BE REPLACED BY AN AGENT ##
type_query = "all"
query = "RAG"
start = 0
max_results = 10
sortby = "submittedDate"
sortorder = "descending"
### SE DEBE CONSTRUIR EL HTML DEPENDIENDO DE LA FUENTE Y ORGANIZAR EN DICCIONARIOS LAS FUENTES ASÍ COMO LAS PALABRAS CLAVE###
url = f'http://export.arxiv.org/api/query?search_query={type_query}:{query}&start={start}&max_results={max_results}&sortBy={sortby}&sortOrder={sortorder}'
#url = f"https://towardsdatascience.com/search?q={query}"

# Let's first define the assistant agent that suggests tool calls.
assistant = ConversableAgent(
    name="Assistant",
    system_message="You are a helpful AI assistant."
    "You can help with web scraping"
    "Please scrape the given link. You won't scrape twice."
    "Return 'TERMINATE' when the task is done.",
    #llm_config={"config_list": [{"model": "gpt-4o", "api_key": os.environ["OPENAI_API_KEY"]}]},
    llm_config={"config_list": [{"model": "llama3.1:latest", "api_type": "ollama", "client_host":"http://localhost:11434"}]},
)

# The user proxy agent is used for interacting with the assistant agent
# and executes tool calls.
user_proxy = ConversableAgent(
    name="User",
    llm_config=False,
    is_termination_msg=lambda msg: msg.get("content") is not None and "TERMINATE" in msg["content"],
    human_input_mode="NEVER",
)

# Register the tool signature with the assistant agent.
assistant.register_for_llm(name="web_scraper_agent", description="A web scraping tool")(fetch_html)

# Register the tool function with the user proxy agent.
user_proxy.register_for_execution(name="web_scraper_agent")(fetch_html)

chat_result = user_proxy.initiate_chat(assistant, message=f"What are ALL the raw pdf links and titles present in {url}?. Just give the link and title, not markdown links.")

User (to Assistant):

What are ALL the raw pdf links and titles present in http://export.arxiv.org/api/query?search_query=all:RAG&start=0&max_results=10&sortBy=submittedDate&sortOrder=descending?. Just give the link and title, not markdown links.

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Assistant (to User):


***** Suggested tool call (ollama_func_6408): web_scraper_agent *****
Arguments: 
{"url": "http://export.arxiv.org/api/query?search_query=all:RAG&start=0&max_results=10&sortBy=submittedDate&sortOrder=descending"}
*********************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION web_scraper_agent...
User (to Assistant):

***** Response from calling tool (ollama_func_6408) *****
<?xml version="1.0" encoding="UTF-8"?>
<feed xmlns="http://www.w3.org/2005/Atom">
  <link href="http://arxiv.org/api/query?

In [8]:
chat_result.cost

{'usage_including_cached_inference': {'total_cost': 0.024127499999999996,
  'gpt-4o-2024-08-06': {'cost': 0.024127499999999996,
   'prompt_tokens': 7891,
   'completion_tokens': 440,
   'total_tokens': 8331}},
 'usage_excluding_cached_inference': {'total_cost': 0.024127499999999996,
  'gpt-4o-2024-08-06': {'cost': 0.024127499999999996,
   'prompt_tokens': 7891,
   'completion_tokens': 440,
   'total_tokens': 8331}}}

In [8]:
print(chat_result.chat_history[-1]["content"])

Advanced System Integration: Analyzing OpenAPI Chunking for Retrieval-Augmented Generation
CantorNet: A Sandbox for Testing Geometrical and Topological Complexity Measures
TERMINATE


In [8]:
"""params = {
  "api_key": os.environ["GOOGLE_SCHOLAR_API_KEY"],
  "engine": "google_scholar",
  "q": "RAG AI",
  "hl": "en"
}

search = GoogleSearch(params)
results = search.get_dict()
results"""

'params = {\n  "api_key": os.environ["GOOGLE_SCHOLAR_API_KEY"],\n  "engine": "google_scholar",\n  "q": "RAG AI",\n  "hl": "en"\n}\n\nsearch = GoogleSearch(params)\nresults = search.get_dict()\nresults'

[[Volver]](#Contenido)

## <span style="color:blue">Extraer Metadata</span>

Texto de la sección X

In [103]:
soup = BeautifulSoup(html, "lxml-xml")
articles = soup.find_all("entry")

data_json = []
for entry in articles:
    prim_category = entry.find("primary_category").get("term")
    published = entry.find("published").text
    updated = entry.find("updated").text
    title = entry.find("title").text
    summary = entry.find("summary").text
    authors = entry.find_all("author")
    authors = [auth.find("name").text for auth in authors]
    link_article = entry.select('link[title="pdf"]')[0].get("href")
    data_json.append({"primary_category": prim_category, 
                      "published": published,
                      "updated": updated,
                      "title": title, 
                      "summary": summary,
                      "authors": authors, 
                      "link_article": link_article})

[[Volver]](#Contenido)

## <span style="color:blue">Convertir en DataFrame</span>

Texto de la sección X

In [104]:
data_df = pd.DataFrame.from_dict(data_json)
columns_to_convert = ['published', 'updated']
data_df[columns_to_convert] = data_df[columns_to_convert].apply(pd.to_datetime)
data_df.insert(0, "id", data_df["link_article"].str.split(".").str[-1].str.replace("v*","", regex=True))
data_df["id"] = data_df["id"].astype("int")
data_df.insert(1, "version", data_df["link_article"].str.split(".").str[-1].str.split("v").str[-1])
data_df.info()
data_df.sort_values(by="published", ascending=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype              
---  ------            --------------  -----              
 0   id                20 non-null     int64              
 1   version           20 non-null     object             
 2   primary_category  20 non-null     object             
 3   published         20 non-null     datetime64[ns, UTC]
 4   updated           20 non-null     datetime64[ns, UTC]
 5   title             20 non-null     object             
 6   summary           20 non-null     object             
 7   authors           20 non-null     object             
 8   link_article      20 non-null     object             
dtypes: datetime64[ns, UTC](2), int64(1), object(6)
memory usage: 1.5+ KB


,id,version,primary_category,published,updated,title,summary,authors,link_article
0,198041,1,cs.SE,2024-11-29 16:09:43+00:00,2024-11-29 16:09:43+00:00,Advanced System Integration: Analyzing OpenAPI...,Integrating multiple (sub-)systems is essent...,"[Robin D. Pesl, Jerin G. Mathew, Massimo Mecel...",http://arxiv.org/pdf/2411.19804v1
1,197131,1,cs.NE,2024-11-29 14:01:34+00:00,2024-11-29 14:01:34+00:00,CantorNet: A Sandbox for Testing Topological a...,Many natural phenomena are characterized by ...,"[Michal Lewandowski, Hamid Eghbalzadeh, Bernha...",http://arxiv.org/pdf/2411.19713v1
2,197101,1,cs.IR,2024-11-29 13:57:07+00:00,2024-11-29 13:57:07+00:00,Know Your RAG: Dataset Taxonomy and Generation...,Retrieval Augmented Generation (RAG) systems...,"[Rafael Teixeira de Lima, Shubham Gupta, Cesar...",http://arxiv.org/pdf/2411.19710v1
3,195541,1,cs.HC,2024-11-29 09:07:21+00:00,2024-11-29 09:07:21+00:00,Unimib Assistant: designing a student-friendly...,Natural language processing skills of Large ...,"[Chiara Antico, Stefano Giordano, Cansu Koyutu...",http://arxiv.org/pdf/2411.19554v1
4,195391,1,cs.AI,2024-11-29 08:34:07+00:00,2024-11-29 08:34:07+00:00,Knowledge Management for Automobile Failure An...,This paper presents a knowledge management s...,"[Yuta Ojima, Hiroki Sakaji, Tadashi Nakamura, ...",http://arxiv.org/pdf/2411.19539v1
5,195281,1,cs.CV,2024-11-29 07:57:32+00:00,2024-11-29 07:57:32+00:00,RAGDiffusion: Faithful Cloth Generation via Ex...,Standard clothing asset generation involves ...,"[Xianfeng Tan, Yuhan Li, Wenxiang Shang, Yubo ...",http://arxiv.org/pdf/2411.19528v1
6,194631,1,cs.SE,2024-11-29 04:25:31+00:00,2024-11-29 04:25:31+00:00,Towards Understanding Retrieval Accuracy and P...,Retrieval-Augmented Generation (RAG) is a pi...,"[Shengming Zhao, Yuheng Huang, Jiayang Song, Z...",http://arxiv.org/pdf/2411.19463v1
7,194431,1,cs.CL,2024-11-29 03:01:05+00:00,2024-11-29 03:01:05+00:00,Auto-RAG: Autonomous Retrieval-Augmented Gener...,Iterative retrieval refers to the process in...,"[Tian Yu, Shaolei Zhang, Yang Feng]",http://arxiv.org/pdf/2411.19443v1
8,192291,1,cs.HC,2024-11-28 15:53:27+00:00,2024-11-28 15:53:27+00:00,Habit Coach: Customising RAG-based chatbots to...,This paper presents the iterative developmen...,"[Arian Fooroogh Mand Arabi, Cansu Koyuturk, Mi...",http://arxiv.org/pdf/2411.19229v1
9,189481,1,cs.CR,2024-11-28 06:29:46+00:00,2024-11-28 06:29:46+00:00,Knowledge Database or Poison Base? Detecting R...,As Large Language Models (LLMs) are progress...,"[Xue Tan, Hao Luan, Mingyu Luo, Xiaoyan Sun, P...",http://arxiv.org/pdf/2411.18948v1


[[Volver]](#Contenido)

### <span style="color:#4CC9F0">Subsección X</span>

Texto de la subsección X

In [ ]:
# Código de la subsección X

## <span style="color:blue">Automatización</span>

Automatización del proceso

## <span style="color:blue">Uso de la Automatización</span>

Uso de la Automatización explicada

## <span style="color:blue">Conclusiones</span>

Conclusiones del Notebook

## <span style="color:blue">Recomendaciones</span>

Recomendaciones del estudio hecho.

## <span style="color:blue">Referencias</span>

1. [Referencia]()